In [ ]:
import pandas
from googleapiclient.discovery import build

with open('api_key.txt') as f:
    api_key = f.readline()

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
comment = list()

# Create google api object
api_obj = build('youtube', 'v3', developerKey=api_key)
response = api_obj.commentThreads().list(part='snippet,replies', videoId='kTl4lCpej9k', maxResults=100).execute()